# 0. Librerias

In [2]:
# Genérica
# -----------------------------------------------------------------------------
from time import time
import session_info


# Tratamientos datos
# -----------------------------------------------------------------------------
import pandas as pd
import numpy as np


# category encoders
# -----------------------------------------------------------------------------
from category_encoders import OrdinalEncoder


# scikit-learn
# -----------------------------------------------------------------------------
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler


# Optuna
# -----------------------------------------------------------------------------
import optuna
optuna.logging.set_verbosity(optuna.logging.FATAL)


# Gráficos
# -----------------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt


session_info.show(html=False)

-----
category_encoders   2.6.1
matplotlib          3.7.2
numpy               1.23.5
optuna              3.2.0
pandas              1.5.3
seaborn             0.12.2
session_info        1.0.0
sklearn             1.3.0
-----
IPython             8.14.0
jupyter_client      8.3.0
jupyter_core        5.3.1
notebook            6.5.4
-----
Python 3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]
Linux-5.15.0-82-generic-x86_64-with-glibc2.31
-----
Session information updated at 2023-09-04 13:11


In [3]:
# Funciones
# -----------------------------------------------------------------------------
def your_function(xxx):
    pass

# 1. Cargar datos

In [39]:
# Ruta
path = './data/tablon-sell-in-2.csv'


# Cargar datos
df = pd.read_csv(
    path,
    sep=';',
    dtype={
        'Epoca_Covid': 'category',
        'Anio': 'category',
        'Trimestre': 'category',
        'Mes': 'category',
        'Cliente': 'category',
        'Tipo_Cliente': 'category',
        'Distribuidor_MSM': 'category',
        'Codigo_Postal': 'category',
        'Area_Dist': 'category',
        'Zona': 'category',
        'ID_CCAA': 'category',
        'ID_Provincia': 'category',
        'Gama': 'category',
        'Formato': 'category'
    })


# Castear tipo de dato
fcols = df.select_dtypes('float').columns
icols = df.select_dtypes('integer').columns

df[fcols] = df[fcols].apply(pd.to_numeric, downcast='float')
df[icols] = df[icols].apply(pd.to_numeric, downcast='integer')


# Eliminar año 2020
filtro = (df.Anio != '2020')
# filtro = (df.Anio == '2022') & (df.Tipo_Cliente == 'CON')
# filtro = (df.Cliente == '542516')
df = df[filtro]


df.head()

,Fecha,Epoca_Covid,Anio,Trimestre,Mes,Cliente,Tipo_Cliente,Distribuidor_MSM,Codigo_Postal,Area_Dist,...,IPRI,Temperatura_Minima,Temperatura_Media,Temperatura_Maxima,Precipitacion,Sell_In_3,Sell_In_2,Sell_In_1,Sell_In,STRATIO_CREATED_DATE
0,201810,pre-covid,2018,T4,10,558843,DHC,0,43202,ZZ,...,102.220001,12.98,18.32,23.650000,275.600006,0.630000,0.320000,0.55,0.630000,2023-09-01T14:08:41.115Z
1,201810,pre-covid,2018,T4,10,558843,DHC,0,43202,ZZ,...,102.220001,12.98,18.32,23.650000,275.600006,0.550000,0.320000,0.48,0.240000,2023-09-01T14:08:41.115Z
2,201810,pre-covid,2018,T4,10,558850,CON,0,49220,Y4,...,102.620003,7.46,14.08,20.709999,19.040001,39.599998,109.800003,0.00,34.200001,2023-09-01T14:08:41.115Z
3,201810,pre-covid,2018,T4,10,558850,CON,0,49220,Y4,...,102.620003,7.46,14.08,20.709999,19.040001,46.200001,63.000000,0.00,42.000000,2023-09-01T14:08:41.115Z
4,201810,pre-covid,2018,T4,10,558850,CON,0,49220,Y4,...,102.620003,7.46,14.08,20.709999,19.040001,0.000000,11.640000,0.00,0.000000,2023-09-01T14:08:41.115Z


In [40]:
# Columnas del dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1641192 entries, 0 to 1975967
Data columns (total 54 columns):
 #   Column                      Non-Null Count    Dtype   
---  ------                      --------------    -----   
 0   Fecha                       1641192 non-null  int32   
 1   Epoca_Covid                 1641192 non-null  category
 2   Anio                        1641192 non-null  category
 3   Trimestre                   1641192 non-null  category
 4   Mes                         1641192 non-null  category
 5   Cliente                     1641192 non-null  category
 6   Tipo_Cliente                1641192 non-null  category
 7   Distribuidor_MSM            1641192 non-null  category
 8   Codigo_Postal               1641192 non-null  category
 9   Area_Dist                   1641192 non-null  category
 10  Zona                        1641192 non-null  category
 11  ID_CCAA                     1641192 non-null  category
 12  ID_Provincia                1641192 non-nu

In [41]:
# Eliminar columnas
df = df.drop(columns=[
    'Fecha',
    'Anio',
    'Cliente',
    'Frontur',
    'Temperatura_Minima',
    'Temperatura_Maxima',
    'Movimientos_Entrada',
    'Movimientos_Salida',
    'Volumen_Solicitado',
    'STRATIO_CREATED_DATE'
])

# Eliminar registros con NaN
df = df.dropna()

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1641190 entries, 0 to 1975967
Data columns (total 44 columns):
 #   Column                      Non-Null Count    Dtype   
---  ------                      --------------    -----   
 0   Epoca_Covid                 1641190 non-null  category
 1   Trimestre                   1641190 non-null  category
 2   Mes                         1641190 non-null  category
 3   Tipo_Cliente                1641190 non-null  category
 4   Distribuidor_MSM            1641190 non-null  category
 5   Codigo_Postal               1641190 non-null  category
 6   Area_Dist                   1641190 non-null  category
 7   Zona                        1641190 non-null  category
 8   ID_CCAA                     1641190 non-null  category
 9   ID_Provincia                1641190 non-null  category
 10  Gama                        1641190 non-null  category
 11  Formato                     1641190 non-null  category
 12  Num_PdV_CI                  1641190 non-nu

In [43]:
# Ordinal Encoder
categorical_features = [col for col in df.columns if df[col].dtype == 'category']

encoder = OrdinalEncoder(
    cols=categorical_features,
    handle_unknown='ignore',
    return_df=True).fit(df)

df=encoder.transform(df)

df.head()

,Epoca_Covid,Trimestre,Mes,Tipo_Cliente,Distribuidor_MSM,Codigo_Postal,Area_Dist,Zona,ID_CCAA,ID_Provincia,...,Movimientos_Neto,IPC,IPI,IPRI,Temperatura_Media,Precipitacion,Sell_In_3,Sell_In_2,Sell_In_1,Sell_In
0,1,1,1,1,1,1,1,1,1,1,...,53148.5,0.8,118.720001,102.220001,18.32,275.600006,0.630000,0.320000,0.55,0.630000
1,1,1,1,1,1,1,1,1,1,1,...,53148.5,0.8,118.720001,102.220001,18.32,275.600006,0.550000,0.320000,0.48,0.240000
2,1,1,1,2,1,2,2,2,2,2,...,5483.5,1.2,109.089996,102.620003,14.08,19.040001,39.599998,109.800003,0.00,34.200001
3,1,1,1,2,1,2,2,2,2,2,...,5483.5,1.2,109.089996,102.620003,14.08,19.040001,46.200001,63.000000,0.00,42.000000
4,1,1,1,2,1,2,2,2,2,2,...,5483.5,1.2,109.089996,102.620003,14.08,19.040001,0.000000,11.640000,0.00,0.000000


# 2. Modelos

## 2.1. Sin feature engineering

### 2.1.1. Modelo base

In [44]:
# Variable dependiente
target = 'Sell_In'

# Variables independientes
features = list(df.columns)
features.remove(target)

# Mostrar
print(f'Variable dependiente: {target}')
print(30*'-')
print(f'Variable independiente: {features}')

Variable dependiente: Sell_In
------------------------------
Variable independiente: ['Epoca_Covid', 'Trimestre', 'Mes', 'Tipo_Cliente', 'Distribuidor_MSM', 'Codigo_Postal', 'Area_Dist', 'Zona', 'ID_CCAA', 'ID_Provincia', 'Gama', 'Formato', 'Num_PdV_CI', 'Num_PdV_RU', 'Num_PdV_TU', 'Impactos_TV', 'Impactos_OOH', 'Impresiones_Meta', 'Impresiones_Pint', 'Impresiones_Digi', 'Sell_In_PP', 'Marketing_Nacional', 'Trade_Marketing', 'Marketing_PLV', 'Distancia_CdP', 'Dispersion_PdV', 'Volumen_Entregado_Perfecto', 'Nivel_Servicio', 'Tarifa_Eur_Hl', 'Margen_Distribuidor', 'Num_Festivos', 'Num_Festivos_AI', 'Egatur', 'Familitur', 'Movimientos_Neto', 'IPC', 'IPI', 'IPRI', 'Temperatura_Media', 'Precipitacion', 'Sell_In_3', 'Sell_In_2', 'Sell_In_1']


In [45]:
# Seed
seed = 123

# Separar conjunto de test y entrenamiento
train, test = train_test_split(df, train_size=0.95, test_size=0.05, random_state=seed)

In [49]:
# Instanciar modelo
mrfr = RandomForestRegressor(
    # Número de árboles del bosque
    n_estimators = 1000, 
    # Máxima profundidad de cada árbol
    max_depth = 8,
    # Número mínimo de registros en un nodo para separarlo
    min_samples_split= 5,
    # Número mínimo de registros que puede haber en un nodo final (hoja)
    min_samples_leaf = 5,
    # Porcentaje de variables a analizar para buscar la mejor partición ('sqrt', 'log2', 1, float..)
    max_features = 0.8,
    # Bootstrap
    bootstrap = True,
    # Número de jobs en paralelo
    n_jobs = -1,
    # Controla la aleatoriedad del bootstrapping y de las variables de cada árbol
    random_state = seed,
    # Función para medir la calidad de cada división ('absolute_error' --> slower, 'squared_error', ...)
    criterion = 'squared_error'
    )

In [50]:
# Pipeline
model_pipeline = Pipeline(steps=[("mrfr", mrfr)])

In [51]:
# Medir tiempo
tiempo_inicial = time()

# Entrenar modelo
model = model_pipeline.fit(train[features], train[target])

# Medir tiempo
tiempo_final = time()
tiempo_ejecucion = tiempo_final - tiempo_inicial

print('')
print(f'El tiempo de ejecución es de {tiempo_ejecucion/60} minutos')


El tiempo de ejecución es de 55.34075537522634 minutos


In [52]:
# Cálculo de métricas
train_y_real = train[target]
train_y_pred = model.predict(train[features])

test_y_real = test[target]
test_y_pred = model.predict(test[features])

metric_test_r2 = round(r2_score(test_y_real, test_y_pred), 3)
metric_adj_test_r2 = round(1-(1-metric_test_r2)*(len(test)-1)/(len(test)-len(features)-1), 3)

metric_train_mae = round(mean_absolute_error(train_y_real, train_y_pred), 3)
metric_test_mae = round(mean_absolute_error(test_y_real, test_y_pred), 3)

metric_train_rmse = round(mean_squared_error(train_y_real, train_y_pred, squared=False), 3)
metric_test_rmse = round(mean_squared_error(test_y_real, test_y_pred, squared=False), 3)

metric_train_score = round(1.15 - (metric_train_mae / train_y_real.mean()), 3)
metric_test_score = round(1.15 - (metric_test_mae / test_y_real.mean()), 3)


# Mostrar
print('R2 Score')
print(50*'-')
print(f'Test R2: {metric_test_r2}')
print(f'Test R2 Adj: {metric_adj_test_r2}')
print('')

print('Score')
print(50*'-')
print(f'Train: {metric_train_score}')
print(f'Test: {metric_test_score}')
print('')

print('MAE')
print(50*'-')
print(f'Train: {metric_train_mae}')
print(f'Test: {metric_test_mae}')
print('')

print('RMSE')
print(50*'-')
print(f'Train: {metric_train_rmse}')
print(f'Test: {metric_test_rmse}')
print('')

R2 Score
--------------------------------------------------
Test R2: 0.931
Test R2 Adj: 0.931

Score
--------------------------------------------------
Train: 0.876
Test: 0.86

MAE
--------------------------------------------------
Train: 5.267
Test: 5.237

RMSE
--------------------------------------------------
Train: 32.489
Test: 35.671



In [53]:
feature_imp = pd.DataFrame(sorted(zip(features, model[0].feature_importances_)), columns=['Feature', 'Value'])
feature_imp.sort_values(by=['Value'], ascending=False)

,Feature,Value
32,Sell_In_1,0.720829
33,Sell_In_2,0.155565
35,Sell_In_PP,0.055432
34,Sell_In_3,0.023010
22,Marketing_PLV,0.011214
39,Trade_Marketing,0.009006
21,Marketing_Nacional,0.006195
41,Volumen_Entregado_Perfecto,0.005221
24,Movimientos_Neto,0.001329
36,Tarifa_Eur_Hl,0.001246


### 2.1.1. Modelo optimizado

In [54]:
# Variable dependiente
target = 'Sell_In'

X = df.drop(target, axis=1)
y = df[target]

In [91]:
# Intanciar regresor base
regressor_obj = RandomForestRegressor(
    n_estimators=100,
    bootstrap=True,
    n_jobs=-1,
    random_state=seed
)

# Sugerir parámetros a optimizar
param_distributions = {
    "max_depth": optuna.distributions.IntDistribution(2, 16, step=1, log=False),
    "min_samples_split": optuna.distributions.IntDistribution(2, 15, step=1, log=False),
    "min_samples_leaf": optuna.distributions.IntDistribution(2, 15, step=1, log=False),
    "max_features": optuna.distributions.FloatDistribution(0.5, 1.0, step=0.1, log=False)
}

# Instanciar optimizador
optuna_search = optuna.integration.OptunaSearchCV(
    regressor_obj,
    param_distributions,
    # Cross Validation
    cv=5,
    # Number of trials. This trades off runtime vs quality of the solution.
    n_trials=200,
    # Time limit in seconds for the search of appropriate models. This trades off runtime vs quality of the solution.
    timeout=1200,
    verbose=-1
)

/tmp/ipykernel_18068/1230226749.py:18: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(


In [92]:
# Medir tiempo
tiempo_inicial = time()


# Ejecutar optimizador
optuna_search.fit(X, y)


# Medir tiempo
tiempo_final = time()
tiempo_ejecucion = tiempo_final - tiempo_inicial

print('')
print(f'El tiempo de ejecución es de {tiempo_ejecucion/60} minutos')


El tiempo de ejecución es de 17.01727542479833 minutos


In [93]:
# Mostrar score_r2 y los mejores hiperparámetros
print("Best score:", optuna_search.best_score_)
print("Best parameters:", optuna_search.best_params_)

Best score: 0.8799146018836155
Best parameters: {'max_depth': 14, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_features': 0.6}


In [94]:
# Instanciar modelo optimizado
mrfr = RandomForestRegressor(
    n_estimators=1000, 
    max_depth=optuna_search.best_params_['max_depth'],
    min_samples_split=optuna_search.best_params_['min_samples_split'],
    min_samples_leaf=optuna_search.best_params_['min_samples_leaf'],
    max_features=optuna_search.best_params_['max_features'],
    bootstrap = True,
    n_jobs = -1,
    random_state = seed
    )

# Pipeline
model_pipeline = Pipeline(steps=[("mrfr", mrfr)])

# Entrenar modelo
model = model_pipeline.fit(train[features], train[target])

# Cálculo de métricas
metric_test_r2 = round(r2_score(test[target], model.predict(test[features])), 3)
metric_adj_test_r2 = round(1-(1-metric_test_r2)*(len(test)-1)/(len(test)-len(features)-1), 3)

metric_train_mae = round(mean_absolute_error(train[target], model.predict(train[features])), 3)
metric_test_mae = round(mean_absolute_error(test[target], model.predict(test[features])), 3)

metric_train_rmse = round(mean_squared_error(train[target], model.predict(train[features]), squared=False), 3)
metric_test_rmse = round(mean_squared_error(test[target], model.predict(test[features]), squared=False), 3)


# Mostrar
print('R2 Score')
print(50*'-')
print(f'Test R2: {metric_test_r2}')
print(f'Test R2 Adj: {metric_adj_test_r2}')
print('')

print('MAE')
print(50*'-')
print(f'Train: {metric_train_mae}')
print(f'Test: {metric_test_mae}')
print('')

print('RMSE')
print(50*'-')
print(f'Train: {metric_train_rmse}')
print(f'Test: {metric_test_rmse}')
print('')

R2 Score
--------------------------------------------------
Test R2: 0.944
Test R2 Adj: 0.942

MAE
--------------------------------------------------
Train: 46.349
Test: 58.045

RMSE
--------------------------------------------------
Train: 159.208
Test: 202.74



In [21]:
model

Pipeline(steps=[('mrfr',
                 RandomForestRegressor(max_depth=10, max_features=0.9,
                                       min_samples_leaf=2, n_estimators=1000,
                                       n_jobs=-1, random_state=123))])

In [22]:
feature_imp = pd.DataFrame(sorted(zip(features, model[0].feature_importances_)), columns=['Feature', 'Value'])
feature_imp.sort_values(by=['Value'], ascending=False)

,Feature,Value
4,Formato,0.214920
10,Impactos_TV,0.144257
5,Gama,0.132326
21,Tarifa_Eur_Hl,0.116590
2,Egatur,0.080073
17,Num_PdV_CI,0.058439
18,Num_PdV_RU,0.052266
1,Codigo_Postal,0.051170
19,Num_PdV_TU,0.045665
7,ID_Provincia,0.018290


In [23]:
test

,Epoca_Covid,Trimestre,Mes,Codigo_Postal,Area_Dist,Zona,ID_CCAA,ID_Provincia,Gama,Formato,...,Impresiones_Pint,Impresiones_Digi,Num_Festivos,Egatur,IPC,Temperatura_Media,Precipitacion,Movimientos_Neto,Tarifa_Eur_Hl,Sell_In
5064,2,4,9,10,4,5,4,5,12,2,...,0.0,0.0,1.000000,805.66,0.7,20.210918,182.982560,-9.636300e+04,659.41850,0.0000
14809,1,1,12,14,7,9,8,9,3,3,...,0.0,0.0,1.000000,2775.53,0.0,25.863153,58.831350,-1.678304e+06,391.72858,1.1000
10332,2,4,10,15,7,10,9,10,13,3,...,0.0,0.0,0.500000,738.32,0.5,23.419462,16.783485,1.099170e+05,295.28420,71.2600
18898,1,4,8,1,1,1,1,1,3,1,...,0.0,0.0,0.000000,774.47,0.7,13.192160,59.477000,-3.843602e+03,474.97284,7.2000
4548,2,3,7,2,1,1,1,1,13,6,...,0.0,0.0,0.000000,642.18,2.8,10.024116,90.590740,-2.956790e+05,348.10406,7.1064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19413,1,4,10,1,1,1,1,1,1,1,...,0.0,0.0,0.000000,939.51,0.2,21.998806,22.555645,-6.887288e+03,300.03888,16.2000
13895,2,3,5,2,1,1,1,1,13,6,...,0.0,0.0,1.000000,879.23,0.0,6.189963,10.581779,-5.825870e+05,376.56525,15.7248
4196,2,3,6,1,1,1,1,1,10,6,...,0.0,0.0,0.000000,402.27,0.6,10.077687,6.119950,-4.261990e+05,418.31668,1020.4800
4606,2,4,8,5,1,1,1,1,13,3,...,0.0,0.0,2.000000,832.50,-0.2,11.622316,58.977234,-1.333898e+06,295.72595,4.0000


In [24]:
df_test_pred = test.copy()
df_test_pred['Sell_In_Pred'] = model.predict(test[features])

df_test_pred[['Epoca_Covid', 'Trimestre', 'Mes', 'ID_CCAA', 'ID_Provincia', 'Gama', 'Formato', 'Sell_In', 'Sell_In_Pred']]

,Epoca_Covid,Trimestre,Mes,ID_CCAA,ID_Provincia,Gama,Formato,Sell_In,Sell_In_Pred
5064,2,4,9,4,5,12,2,0.0000,1.147909
14809,1,1,12,8,9,3,3,1.1000,1.091106
10332,2,4,10,9,10,13,3,71.2600,22.938859
18898,1,4,8,1,1,3,1,7.2000,8.314597
4548,2,3,7,1,1,13,6,7.1064,17.153003
...,...,...,...,...,...,...,...,...,...
19413,1,4,10,1,1,1,1,16.2000,6.624023
13895,2,3,5,1,1,13,6,15.7248,23.271633
4196,2,3,6,1,1,10,6,1020.4800,954.531441
4606,2,4,8,1,1,13,3,4.0000,6.229651


## 2.2. Eliminando variables correlacionadas

In [59]:
# Ruta
path = './data/tablon-sell-in-2.csv'


# Cargar datos
df = pd.read_csv(
    path,
    sep=';',
    dtype={
        'Epoca_Covid': 'category',
        'Anio': 'category',
        'Trimestre': 'category',
        'Mes': 'category',
        'Cliente': 'category',
        'Tipo_Cliente': 'category',
        'Distribuidor_MSM': 'category',
        'Codigo_Postal': 'category',
        'Area_Dist': 'category',
        'Zona': 'category',
        'ID_CCAA': 'category',
        'ID_Provincia': 'category',
        'Gama': 'category',
        'Formato': 'category'
    })


# Castear tipo de dato
fcols = df.select_dtypes('float').columns
icols = df.select_dtypes('integer').columns

df[fcols] = df[fcols].apply(pd.to_numeric, downcast='float')
df[icols] = df[icols].apply(pd.to_numeric, downcast='integer')


# Eliminar año 2020
filtro = (df.Anio != '2020')
# filtro = (df.Anio == '2022') & (df.Tipo_Cliente == 'CON')
# filtro = (df.Cliente == '542516')
df = df[filtro]


df.head()

,Fecha,Epoca_Covid,Anio,Trimestre,Mes,Cliente,Tipo_Cliente,Distribuidor_MSM,Codigo_Postal,Area_Dist,...,IPRI,Temperatura_Minima,Temperatura_Media,Temperatura_Maxima,Precipitacion,Sell_In_3,Sell_In_2,Sell_In_1,Sell_In,STRATIO_CREATED_DATE
0,201810,pre-covid,2018,T4,10,558843,DHC,0,43202,ZZ,...,102.220001,12.98,18.32,23.650000,275.600006,0.630000,0.320000,0.55,0.630000,2023-09-01T14:08:41.115Z
1,201810,pre-covid,2018,T4,10,558843,DHC,0,43202,ZZ,...,102.220001,12.98,18.32,23.650000,275.600006,0.550000,0.320000,0.48,0.240000,2023-09-01T14:08:41.115Z
2,201810,pre-covid,2018,T4,10,558850,CON,0,49220,Y4,...,102.620003,7.46,14.08,20.709999,19.040001,39.599998,109.800003,0.00,34.200001,2023-09-01T14:08:41.115Z
3,201810,pre-covid,2018,T4,10,558850,CON,0,49220,Y4,...,102.620003,7.46,14.08,20.709999,19.040001,46.200001,63.000000,0.00,42.000000,2023-09-01T14:08:41.115Z
4,201810,pre-covid,2018,T4,10,558850,CON,0,49220,Y4,...,102.620003,7.46,14.08,20.709999,19.040001,0.000000,11.640000,0.00,0.000000,2023-09-01T14:08:41.115Z


In [60]:
# Columnas del dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1641192 entries, 0 to 1975967
Data columns (total 54 columns):
 #   Column                      Non-Null Count    Dtype   
---  ------                      --------------    -----   
 0   Fecha                       1641192 non-null  int32   
 1   Epoca_Covid                 1641192 non-null  category
 2   Anio                        1641192 non-null  category
 3   Trimestre                   1641192 non-null  category
 4   Mes                         1641192 non-null  category
 5   Cliente                     1641192 non-null  category
 6   Tipo_Cliente                1641192 non-null  category
 7   Distribuidor_MSM            1641192 non-null  category
 8   Codigo_Postal               1641192 non-null  category
 9   Area_Dist                   1641192 non-null  category
 10  Zona                        1641192 non-null  category
 11  ID_CCAA                     1641192 non-null  category
 12  ID_Provincia                1641192 non-nu

In [61]:
# Eliminar columnas
df = df.drop(columns=[
    'Fecha',
    'Anio',
    'Cliente',
    'Codigo_Postal',
    'Frontur',
    'Temperatura_Minima',
    'Temperatura_Maxima',
    'Movimientos_Entrada',
    'Movimientos_Salida',
    'Volumen_Solicitado',
    'Volumen_Entregado_Perfecto',
    'Sell_In_PP',
    'Sell_In_3',
    'Sell_In_2',
    'Sell_In_1',
    'STRATIO_CREATED_DATE'
])

# Eliminar registros con NaN
df = df.dropna()

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1641190 entries, 0 to 1975967
Data columns (total 38 columns):
 #   Column               Non-Null Count    Dtype   
---  ------               --------------    -----   
 0   Epoca_Covid          1641190 non-null  category
 1   Trimestre            1641190 non-null  category
 2   Mes                  1641190 non-null  category
 3   Tipo_Cliente         1641190 non-null  category
 4   Distribuidor_MSM     1641190 non-null  category
 5   Area_Dist            1641190 non-null  category
 6   Zona                 1641190 non-null  category
 7   ID_CCAA              1641190 non-null  category
 8   ID_Provincia         1641190 non-null  category
 9   Gama                 1641190 non-null  category
 10  Formato              1641190 non-null  category
 11  Num_PdV_CI           1641190 non-null  float32 
 12  Num_PdV_RU           1641190 non-null  float32 
 13  Num_PdV_TU           1641190 non-null  float32 
 14  Impactos_TV          1641190 non-n

In [63]:
# Ordinal Encoder
categorical_features = [col for col in df.columns if df[col].dtype == 'category']

encoder = OrdinalEncoder(
    cols=categorical_features,
    handle_unknown='ignore',
    return_df=True).fit(df)

df=encoder.transform(df)

df.head()

,Epoca_Covid,Trimestre,Mes,Tipo_Cliente,Distribuidor_MSM,Area_Dist,Zona,ID_CCAA,ID_Provincia,Gama,...,Num_Festivos_AI,Egatur,Familitur,Movimientos_Neto,IPC,IPI,IPRI,Temperatura_Media,Precipitacion,Sell_In
0,1,1,1,1,1,1,1,1,1,1,...,0,1710.040039,0.0,53148.5,0.8,118.720001,102.220001,18.32,275.600006,0.630000
1,1,1,1,1,1,1,1,1,1,1,...,0,1710.040039,0.0,53148.5,0.8,118.720001,102.220001,18.32,275.600006,0.240000
2,1,1,1,2,1,2,2,2,2,1,...,0,96.709999,0.0,5483.5,1.2,109.089996,102.620003,14.08,19.040001,34.200001
3,1,1,1,2,1,2,2,2,2,1,...,0,96.709999,0.0,5483.5,1.2,109.089996,102.620003,14.08,19.040001,42.000000
4,1,1,1,2,1,2,2,2,2,1,...,0,96.709999,0.0,5483.5,1.2,109.089996,102.620003,14.08,19.040001,0.000000


### 2.2.1. Modelo base

In [64]:
# Variable dependiente
target = 'Sell_In'

# Variables independientes
features = list(df.columns)
features.remove(target)

# Mostrar
print(f'Variable dependiente: {target}')
print(30*'-')
print(f'Variable independiente: {features}')

Variable dependiente: Sell_In
------------------------------
Variable independiente: ['Epoca_Covid', 'Trimestre', 'Mes', 'Tipo_Cliente', 'Distribuidor_MSM', 'Area_Dist', 'Zona', 'ID_CCAA', 'ID_Provincia', 'Gama', 'Formato', 'Num_PdV_CI', 'Num_PdV_RU', 'Num_PdV_TU', 'Impactos_TV', 'Impactos_OOH', 'Impresiones_Meta', 'Impresiones_Pint', 'Impresiones_Digi', 'Marketing_Nacional', 'Trade_Marketing', 'Marketing_PLV', 'Distancia_CdP', 'Dispersion_PdV', 'Nivel_Servicio', 'Tarifa_Eur_Hl', 'Margen_Distribuidor', 'Num_Festivos', 'Num_Festivos_AI', 'Egatur', 'Familitur', 'Movimientos_Neto', 'IPC', 'IPI', 'IPRI', 'Temperatura_Media', 'Precipitacion']


In [65]:
# Seed
seed = 123

# Separar conjunto de test y entrenamiento
train, test = train_test_split(df, train_size=0.95, test_size=0.05, random_state=seed)

In [66]:
# Instanciar modelo
mrfr = RandomForestRegressor(
    # Número de árboles del bosque
    n_estimators = 1000, 
    # Máxima profundidad de cada árbol
    max_depth = 8,
    # Número mínimo de registros en un nodo para separarlo
    min_samples_split= 5,
    # Número mínimo de registros que puede haber en un nodo final (hoja)
    min_samples_leaf = 5,
    # Porcentaje de variables a analizar para buscar la mejor partición ('sqrt', 'log2', 1, float..)
    max_features = 0.8,
    # Bootstrap
    bootstrap = True,
    # Número de jobs en paralelo
    n_jobs = -1,
    # Controla la aleatoriedad del bootstrapping y de las variables de cada árbol
    random_state = seed,
    # Función para medir la calidad de cada división ('absolute_error' --> slower, 'squared_error', ...)
    criterion = 'squared_error'
    )

In [67]:
# Pipeline
model_pipeline = Pipeline(steps=[("mrfr", mrfr)])

In [68]:
# Medir tiempo
tiempo_inicial = time()

# Entrenar modelo
model = model_pipeline.fit(train[features], train[target])

# Medir tiempo
tiempo_final = time()
tiempo_ejecucion = tiempo_final - tiempo_inicial

print('')
print(f'El tiempo de ejecución es de {tiempo_ejecucion/60} minutos')


El tiempo de ejecución es de 48.81000893513362 minutos


In [69]:
# Cálculo de métricas
train_y_real = train[target]
train_y_pred = model.predict(train[features])

test_y_real = test[target]
test_y_pred = model.predict(test[features])

metric_test_r2 = round(r2_score(test_y_real, test_y_pred), 3)
metric_adj_test_r2 = round(1-(1-metric_test_r2)*(len(test)-1)/(len(test)-len(features)-1), 3)

metric_train_mae = round(mean_absolute_error(train_y_real, train_y_pred), 3)
metric_test_mae = round(mean_absolute_error(test_y_real, test_y_pred), 3)

metric_train_rmse = round(mean_squared_error(train_y_real, train_y_pred, squared=False), 3)
metric_test_rmse = round(mean_squared_error(test_y_real, test_y_pred, squared=False), 3)

metric_train_score = round(1.00 - (metric_train_mae / train_y_real.mean()), 3)
metric_test_score = round(1.00 - (metric_test_mae / test_y_real.mean()), 3)


# Mostrar
print('R2 Score')
print(50*'-')
print(f'Test R2: {metric_test_r2}')
print(f'Test R2 Adj: {metric_adj_test_r2}')
print('')

print('Score')
print(50*'-')
print(f'Train: {metric_train_score}')
print(f'Test: {metric_test_score}')
print('')

print('MAE')
print(50*'-')
print(f'Train: {metric_train_mae}')
print(f'Test: {metric_test_mae}')
print('')

print('RMSE')
print(50*'-')
print(f'Train: {metric_train_rmse}')
print(f'Test: {metric_test_rmse}')
print('')

R2 Score
--------------------------------------------------
Test R2: 0.799
Test R2 Adj: 0.799

Score
--------------------------------------------------
Train: 0.672
Test: 0.637

MAE
--------------------------------------------------
Train: 9.209
Test: 9.25

RMSE
--------------------------------------------------
Train: 57.301
Test: 61.044



In [70]:
feature_imp = pd.DataFrame(sorted(zip(features, model[0].feature_importances_)), columns=['Feature', 'Value'])
feature_imp.sort_values(by=['Value'], ascending=False)

,Feature,Value
34,Trade_Marketing,0.345869
20,Marketing_Nacional,0.298325
21,Marketing_PLV,0.138097
27,Num_PdV_CI,0.056867
31,Tarifa_Eur_Hl,0.030623
29,Num_PdV_TU,0.026090
19,Margen_Distribuidor,0.022004
0,Area_Dist,0.017981
33,Tipo_Cliente,0.016490
4,Egatur,0.005478


In [ ]:
# Generamos 3 columnas adicionales con el dato de Sell_In del mes n-1, n-2 y n-3
df['Sell_In_1']= df.sort_values(by=['Distribuidor', 'Gama', 'Formato', 'Fecha'], ascending=True) \
                   .groupby(['Distribuidor', 'Gama', 'Formato'])['Sell_In'] \
                   .shift(periods=1, fill_value=0)

df['Sell_In_2']= df.sort_values(by=['Distribuidor', 'Gama', 'Formato', 'Fecha'], ascending=True) \
                   .groupby(['Distribuidor', 'Gama', 'Formato'])['Sell_In'] \
                   .shift(periods=2, fill_value=0)

df['Sell_In_3']= df.sort_values(by=['Distribuidor', 'Gama', 'Formato', 'Fecha'], ascending=True) \
                   .groupby(['Distribuidor', 'Gama', 'Formato'])['Sell_In'] \
                   .shift(periods=3, fill_value=0)